In [ ]:
import pandas as pd
import numpy as np

in_df = pd.read_excel(
    "https://tfl.gov.uk/cdn/static/cms/documents/out-of-station-interchanges.ods",
    storage_options={"User-Agent": "Mozilla/5.0"},
    skiprows=2,
)

in_df

In [ ]:
def splitty_splitter(inString):
    if isinstance(inString, float):
        return inString
    elif " and " in inString:
        return [*inString.split(" and "), "and"]
    elif " to " in inString:
        return [*inString.split(" to "), "to"]
    else:
        raise Exception(f"Failed to split {inString} into two thingies")
    
def capitalism(inString):
    if isinstance(inString, float):
        return inString
    else:
        inString = list(inString)
        inString[0] = inString[0].upper()
        inString = "".join(inString)
        return inString


in_df["interchange_note_split"] = in_df["Interchange between"].apply(splitty_splitter)
in_df["interchange_note_a"] = in_df["interchange_note_split"].apply(
    lambda x: x[0] if not isinstance(x, float) else x
)
in_df["interchange_note_joiner"] = in_df["interchange_note_split"].apply(
    lambda x: x[2] if not isinstance(x, float) else x
)
in_df["interchange_note_b"] = in_df["interchange_note_split"].apply(
    lambda x: x[1] if not isinstance(x, float) else x
)

in_df = in_df.drop(
    [
        "interchange_note_split",
        "Interchange between",
        "Station_A",
        "Station_B",
        "SameNLCMagneticTime",
    ],
    axis=1,
)

in_df

In [ ]:
concat_df = pd.concat(
    [
        in_df[
            [
                "Station_A_Name",
                "Station_B_Name",
                "InterchangeTimeAB",
                "Type",
                "interchange_note_a",
                "interchange_note_joiner",
                "interchange_note_b",
            ]
        ].rename(
            {
                "Station_A_Name": "from",
                "Station_B_Name": "to",
                "InterchangeTimeAB": "time",
                "Type": "type",
                "interchange_note_a": "interchange_note_0",
                "interchange_note_joiner": "interchange_note_1",
                "interchange_note_b": "interchange_note_2",
            },
            axis=1,
        ),
        in_df[
            [
                "Station_B_Name",
                "Station_A_Name",
                "InterchangeTimeBA",
                "Type",
                "interchange_note_b",
                "interchange_note_joiner",
                "interchange_note_a",
            ]
        ].rename(
            {
                "Station_B_Name": "from",
                "Station_A_Name": "to",
                "InterchangeTimeBA": "time",
                "Type": "type",
                "interchange_note_b": "interchange_note_0",
                "interchange_note_joiner": "interchange_note_1",
                "interchange_note_a": "interchange_note_2",
            },
            axis=1,
        ),
    ],
    axis=0,
).reset_index(drop=True)

concat_df["interchange_note"] = concat_df.apply(
    lambda x: (
        capitalism(" ".join([x.interchange_note_0, x.interchange_note_1, x.interchange_note_2]))
        if not isinstance(x.interchange_note_0, float)
        else np.nan
    ),
    axis=1,
)

concat_df = concat_df.drop(["interchange_note_0", "interchange_note_1", "interchange_note_2"], axis=1)

concat_df

In [ ]:
concat_df.to_json("osi.json", orient="records", indent=4)